# Data aggregation for street - grouping by month and LSOA code, for a selected district (looking on top 5 crime types)

## Import libraries and data

In [ ]:
import pandas as pd
import os
import tqdm as tq
import string
import statistics as stats
import re
pd.set_option('display.max_columns', None)

In [ ]:
# To make a dataframe for street for one district only (district name must correspond to the one in file name)

def extract_street_for_district(district: str):
    directory = "Jan_2010_Oct_2021" # Change directory
    df_street = pd.DataFrame()

    folders = [folder for folder in os.listdir(directory) if not str(folder).strip("'b").startswith('.')]

    for folder in folders:
        files_street = []

        # Here you can change the date range
        if int(folder[:4]) > 2014:
            # generate the path to folder
            folder_direc = os.fsencode(directory + '/'+ str(folder))
        
            # add each file name to the appropriate list
            for file in os.listdir(folder_direc):
                file = str(file).strip("'b")
                # check if the file name contains the appropriate district name and 'street'
                if (bool(re.search(str(district), file))) & (bool(re.search('street', file))):
                        files_street.append(file)
                
            # Clean dataframe
            for file in files_street:
                # Create dataframe from the current file
                current_data_street = pd.read_csv(directory + '/' + str(folder).strip("'b") + '/' + str(file)) 
                # Drop the attributes considered irrelevant
                current_data_street.drop(['Reported by', 'Longitude', 'Latitude', \
                                          'Location', 'Context', 'LSOA name'], axis=1, inplace=True)
                current_data_street.dropna(subset=['LSOA code'], inplace=True)
                current_data_street.dropna(subset=['Crime type'], inplace=True)
                df_street = df_street.append(current_data_street)
    return df_street

In [ ]:
# call the function and save the df with the appropriate district name: 
# df_str_district = extract_street_for_district
df_str_btp = extract_street_for_district("btp")

In [ ]:
# To make a dataframe for stop-and-search for one district only 
# (district name must correspond to the one in file name)

def extract_sas_for_district(district):
    directory = "Jan_2010_Oct_2021" # Change directory
    df_sas = pd.DataFrame()

    folders = [folder for folder in os.listdir(directory) if not str(folder).strip("'b").startswith('.')]
    
    for folder in folders:
        files_sas = []

        # Here you can change the date range
        if int(folder[:4]) > 2014:
            # generate the path to folder
            folder_direc = os.fsencode(directory + '/'+ str(folder))
        
            # add each file name to the appropriate list
            for file in os.listdir(folder_direc):
                file = str(file).strip("'b")
                # check if the file name contains the appropriate district name and 'street'
                if (bool(re.search(str(district), file))) & (bool(re.search('stop-and-search', file))):
                        files_sas.append(file)
                
            # Clean dataframe for stop-and-search
            for file in files_sas:
                # Create dataframe from the current file
                current_data_sas = pd.read_csv(directory + '/' + str(folder).strip("'b") + '/' + str(file))  
                # drop the attributes considered irrelevant
                current_data_sas.drop(['Policing operation', 'Gender', 'Object of search', \
                                      'Outcome linked to object of search', 'Removal of more than just outer clothing',
                                      'Self-defined ethnicity', 'Officer-defined ethnicity'], axis=1, inplace=True)
                # drop rows which do not have the "Type" specified
                current_data_sas.dropna(subset=['Type'], inplace=True)
                current_data_sas.dropna(subset=['Longitude'], inplace=True)
                current_data_sas.dropna(subset=['Latitude'], inplace=True)
                df_sas = df_sas.append(current_data_sas)
    return  df_sas


In [ ]:
# call the function and save the df with the appropriate district name: 
# df_sas_district = extract_sas_for_district
df_sas_btp = extract_sas_for_district("btp")

In [ ]:
df_str_btp

,Crime ID,Month,Falls within,LSOA code,Crime type,Last outcome category
0,NaN,2020-09,British Transport Police,E01031365,Bicycle theft,NaN
1,NaN,2020-09,British Transport Police,E01031365,Criminal damage and arson,NaN
2,NaN,2020-09,British Transport Police,E01031365,Criminal damage and arson,NaN
3,NaN,2020-09,British Transport Police,E01031365,Other theft,NaN
4,NaN,2020-09,British Transport Police,E01031365,Public order,NaN
...,...,...,...,...,...,...
3039,NaN,2020-08,British Transport Police,E01017839,Public order,NaN
3040,NaN,2020-08,British Transport Police,E01017839,Violence and sexual offences,NaN
3041,NaN,2020-08,British Transport Police,E01032466,Criminal damage and arson,NaN
3042,NaN,2020-08,British Transport Police,E01032466,Other theft,NaN


## Identify and filter on top 5 crimes for this police force

In [ ]:
# identify top 5 crimes
df_top_crimes = df_str_btp.groupby('Crime type').count()['LSOA code'].sort_values(ascending=False).reset_index()
top_crimes = [crime_type for crime_type in df_top_crimes['Crime type'][:5]]

# filter data - only keep the top 5 crime types
df_str_btp_top = df_str_btp[df_str_btp['Crime type'].isin(top_crimes)]
# for this example, the numer of entires go from 280k to 215k so not much data is 'lost'

In [ ]:
# one hot encoding - add columns for each crime type
df_str_btp_top = pd.get_dummies(df_str_btp_top, columns = ['Crime type'])

In [ ]:
# gorup by month and LSOA code
df_str_btp_top = df_str_btp_top.groupby(['Month', 'LSOA code']).count()

In [ ]:
# add a column for the sum of the (top 5) crimes in that month/LSOA
df_str_btp_top['sum_crimes'] = df_str_btp_top['Crime type_Bicycle theft'] + \
df_str_btp_top['Crime type_Other theft'] + df_str_btp_top['Crime type_Public order'] +\
df_str_btp_top['Crime type_Theft from the person'] + df_str_btp_top['Crime type_Violence and sexual offences']
# only keep the relevant columns 
df_str_btp_top = df_str_btp_top[['Crime type_Bicycle theft',
                                'Crime type_Other theft',
                                'Crime type_Public order',
                                'Crime type_Theft from the person',
                                'Crime type_Violence and sexual offences',
                                'sum_crimes']]

In [ ]:
df_str_btp_top

Crime type_Bicycle theft  Crime type_Other theft  \
Month   LSOA code                                                     
2017-10 E01000001                         1                       1   
        E01000005                         3                       3   
        E01000007                        20                      20   
        E01000013                         1                       1   
        E01000036                         4                       4   
...                                     ...                     ...   
2020-09 W01001789                         1                       1   
        W01001851                         1                       1   
        W01001922                         1                       1   
        W01001941                         6                       6   
        W01001955                         3                       3   

                   Crime type_Public order  Crime type_Theft from the person  \
Month   LSOA code                                                              
2017-10 E01000001                        1                                 1   
        E01000005                        3                                 3   
        E01000007                       20                                20   
        E01000013                        1                                 1   
        E01000036                        4                                 4   
...                                    ...                               ...   
2020-09 W01001789                        1                                 1   
        W01001851                        1                                 1   
        W01001922                        1                                 1   
        W01001941                        6                                 6   
        W01001955                        3                                 3   

                   Crime type_Violence and sexual offences  sum_crimes  
Month   LSOA code                                                       
2017-10 E01000001                                        1           5  
        E01000005                                        3          15  
        E01000007                                       20         100  
        E01000013                                        1           5  
        E01000036                                        4          20  
...                                                    ...         ...  
2020-09 W01001789                                        1           5  
        W01001851                                        1           5  
        W01001922                                        1           5  
        W01001941                                        6          30  
        W01001955                                        3          15  

[6711 rows x 6 columns]

In [ ]:
df_sas_btp

,Type,Date,Part of a policing operation,Latitude,Longitude,Age range,Legislation,Outcome
0,Person search,2020-09-01T05:30:00+00:00,NaN,51.374900,-0.092657,over 34,Police and Criminal Evidence Act 1984 (section 1),A no further action disposal
1,Person search,2020-09-01T06:48:00+00:00,NaN,52.477800,-1.898950,NaN,Misuse of Drugs Act 1971 (section 23),A no further action disposal
2,Person search,2020-09-01T06:50:00+00:00,NaN,51.532400,0.003815,25-34,Misuse of Drugs Act 1971 (section 23),A no further action disposal
3,Person search,2020-09-01T06:56:00+00:00,NaN,51.518500,-0.722864,25-34,Police and Criminal Evidence Act 1984 (section 1),A no further action disposal
4,Person search,2020-09-01T07:05:00+00:00,NaN,51.532400,0.003815,over 34,Misuse of Drugs Act 1971 (section 23),A no further action disposal
...,...,...,...,...,...,...,...,...
790,Person search,2020-08-31T17:00:00+00:00,NaN,51.504700,-0.218692,over 34,Police and Criminal Evidence Act 1984 (section 1),Summons / charged by post
791,Person search,2020-08-31T17:25:00+00:00,NaN,51.523466,-0.155997,25-34,Police and Criminal Evidence Act 1984 (section 1),A no further action disposal
792,Person search,2020-08-31T18:45:00+00:00,NaN,53.477512,-2.226586,25-34,Police and Criminal Evidence Act 1984 (section 1),A no further action disposal
793,Person search,2020-08-31T18:50:00+00:00,NaN,53.477512,-2.226586,25-34,Police and Criminal Evidence Act 1984 (section 1),A no further action disposal


In [ ]:
# if needed to export to csv, use:
df_str_btp_top.to_csv('btp_str_top_grouped.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=40789b9a-1c62-45b9-9d9c-b1a39ebe3dfd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>